# Notebook used to used to load csv files into Pandas DataFrames, reformat and store.

In [45]:
import pandas as pd
import matplotlib
import datetime as dt
import pickle
import seaborn as sns
sns.set(rc={'figure.figsize':(11.7,8.27)})
import warnings
warnings.filterwarnings('ignore')

# Load the three data files, inspect, merge and split by 2010-11/2012

In [46]:
df_stores = pd.read_csv('../data/original_data/stores data-set.csv')
df_stores.head(2)

,Store,Type,Size
0,1,A,151315
1,2,A,202307


In [47]:
df_stores.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45 entries, 0 to 44
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Store   45 non-null     int64 
 1   Type    45 non-null     object
 2   Size    45 non-null     int64 
dtypes: int64(2), object(1)
memory usage: 1.2+ KB


In [48]:
df_features = pd.read_csv('../data/original_data/Features data set.csv')
df_features.head(2)

,Store,Date,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday
0,1,05/02/2010,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,False
1,1,12/02/2010,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.242170,8.106,True


In [49]:
df_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8190 entries, 0 to 8189
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Store         8190 non-null   int64  
 1   Date          8190 non-null   object 
 2   Temperature   8190 non-null   float64
 3   Fuel_Price    8190 non-null   float64
 4   MarkDown1     4032 non-null   float64
 5   MarkDown2     2921 non-null   float64
 6   MarkDown3     3613 non-null   float64
 7   MarkDown4     3464 non-null   float64
 8   MarkDown5     4050 non-null   float64
 9   CPI           7605 non-null   float64
 10  Unemployment  7605 non-null   float64
 11  IsHoliday     8190 non-null   bool   
dtypes: bool(1), float64(9), int64(1), object(1)
memory usage: 712.0+ KB


## Several columns appear to have null values - fillna with zeros

In [50]:
df_features.fillna(0, inplace=True)

## Convert dates to datetime with format "%d/%m/%Y" 

In [51]:
df_features['Date']= pd.to_datetime(df_features['Date'], format='%d/%m/%Y')

In [52]:
df_sales = pd.read_csv('../data/original_data/sales data-set.csv')
df_sales.drop(["Unnamed: 5", "Unnamed: 6"], axis=1, inplace=True)
df_sales.head(2)

,Store,Dept,Date,Weekly_Sales,IsHoliday
0,1,1,05/02/2010,24924.50,False
1,1,1,12/02/2010,46039.49,True


In [53]:
df_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 421570 entries, 0 to 421569
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Store         421570 non-null  int64  
 1   Dept          421570 non-null  int64  
 2   Date          421570 non-null  object 
 3   Weekly_Sales  421570 non-null  float64
 4   IsHoliday     421570 non-null  bool   
dtypes: bool(1), float64(1), int64(2), object(1)
memory usage: 13.3+ MB


In [54]:
df_sales['Date'] = pd.to_datetime(df_sales["Date"], format="%d/%m/%Y")

In [55]:
df_sales.describe()

,Store,Dept,Weekly_Sales
count,421570.000000,421570.000000,421570.000000
mean,22.200546,44.260317,15981.258123
std,12.785297,30.492054,22711.183519
min,1.000000,1.000000,-4988.940000
25%,11.000000,18.000000,2079.650000
50%,22.000000,37.000000,7612.030000
75%,33.000000,74.000000,20205.852500
max,45.000000,99.000000,693099.360000


## Quick fix to remove negative values from Weekly_Sales data

In [56]:
# num = df_sales._get_numeric_data()
# num[num < 0] = 0

# Merge the 3 DataFrames

In [57]:
df_features_sales = pd.merge(df_features, df_sales, left_on =['Store','Date','IsHoliday'],
                             right_on=['Store','Date','IsHoliday'])

df_final = pd.merge(df_features_sales,df_stores, left_on =['Store'],right_on=['Store'])

In [58]:
df_final.tail()

,Store,Date,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday,Dept,Weekly_Sales,Type,Size
421565,45,2012-10-26,58.85,3.882,4018.91,58.08,100.0,211.94,858.33,192.308899,8.667,False,93,2487.80,B,118221
421566,45,2012-10-26,58.85,3.882,4018.91,58.08,100.0,211.94,858.33,192.308899,8.667,False,94,5203.31,B,118221
421567,45,2012-10-26,58.85,3.882,4018.91,58.08,100.0,211.94,858.33,192.308899,8.667,False,95,56017.47,B,118221
421568,45,2012-10-26,58.85,3.882,4018.91,58.08,100.0,211.94,858.33,192.308899,8.667,False,97,6817.48,B,118221
421569,45,2012-10-26,58.85,3.882,4018.91,58.08,100.0,211.94,858.33,192.308899,8.667,False,98,1076.80,B,118221


# Split into 2010/11 and 2012  - Most EDA performed on 2010/11

In [59]:
df_2010_11 = df_final[df_final['Date'].dt.year != 2012]
df_2010_11.reset_index(inplace=True)
df_2010_11.drop(['index'], axis=1, inplace=True)
df_2010_11.tail(3)

,Store,Date,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday,Dept,Weekly_Sales,Type,Size
294129,45,2011-12-30,37.79,3.389,3604.35,43941.56,287.45,357.43,750.01,189.062016,8.523,True,95,42084.36,B,118221
294130,45,2011-12-30,37.79,3.389,3604.35,43941.56,287.45,357.43,750.01,189.062016,8.523,True,97,5569.82,B,118221
294131,45,2011-12-30,37.79,3.389,3604.35,43941.56,287.45,357.43,750.01,189.062016,8.523,True,98,553.21,B,118221


In [60]:
df_2010_11.to_csv("../data/wrangled_data/df_2010_11.csv")

In [61]:
df_2012 = df_final[df_final['Date'].dt.year == 2012]
df_2012.reset_index(inplace=True)
df_2012.drop(['index'], axis=1, inplace=True)
df_2012.head(3)

,Store,Date,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday,Dept,Weekly_Sales,Type,Size
0,1,2012-01-06,49.01,3.157,6277.39,21813.16,143.1,1450.13,8483.0,219.714258,7.348,False,1,16567.69,A,151315
1,1,2012-01-06,49.01,3.157,6277.39,21813.16,143.1,1450.13,8483.0,219.714258,7.348,False,2,44481.38,A,151315
2,1,2012-01-06,49.01,3.157,6277.39,21813.16,143.1,1450.13,8483.0,219.714258,7.348,False,3,13926.04,A,151315


## Create DF for each Store/Department - Fill in missing dates in DataFrame

In [62]:
%%time
df_dict = {}
for st in df_2010_11['Store'].unique():
    dp_dict = {}
    for dp in df_2010_11['Dept'].unique():
        df_sales_by_store_dept = df_2010_11[(df_2010_11['Store'] == st) & (df_2010_11['Dept'] == dp)]
        df_sales_by_store_dept.index = df_sales_by_store_dept["Date"]
        if len(df_sales_by_store_dept) == 0:
            continue
        if len(df_sales_by_store_dept) < 100:
            df_sales_by_store_dept = df_sales_by_store_dept.groupby('Store').resample('W-FRI').pad()
            df_sales_by_store_dept = df_sales_by_store_dept.droplevel('Store')
        dp_dict[dp] = df_sales_by_store_dept
    df_dict[st] =  dp_dict

Wall time: 15.2 s


In [63]:
with open('../data/wrangled_data/dict_of_dfs_padded.pickle', 'wb') as handle:
    pickle.dump(df_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

## Concat in to one big DataFrame

In [64]:
value_list = []
for k, v in df_dict.items():
    for k1, v1 in v.items():
        value_list.append(v1) 

df = pd.concat(value_list)
df.head(3)

,Store,Date,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday,Dept,Weekly_Sales,Type,Size
Date,,,,,,,,,,,,,,,,
2010-02-05,1,2010-02-05,42.31,2.572,0.0,0.0,0.0,0.0,0.0,211.096358,8.106,False,1,24924.50,A,151315
2010-02-12,1,2010-02-12,38.51,2.548,0.0,0.0,0.0,0.0,0.0,211.242170,8.106,True,1,46039.49,A,151315
2010-02-19,1,2010-02-19,39.93,2.514,0.0,0.0,0.0,0.0,0.0,211.289143,8.106,False,1,41595.55,A,151315


In [65]:
df.to_csv("../data/wrangled_data/df_with_padding.csv")

# Total Sales for store (Departments summed)

In [66]:
df_total_sales = df_2010_11.groupby(['Store', 'Date'])\
                ["Weekly_Sales"].sum().reset_index().join(df_2010_11.drop(\
                ["Date", "Dept", "Weekly_Sales"], axis=1).drop_duplicates().reset_index(),
                 lsuffix='_caller')

In [67]:
df_total_sales.head(4)

,Store_caller,Date,Weekly_Sales,index,Store,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday,Type,Size
0,1,2010-02-05,1643690.90,0,1,42.31,2.572,0.0,0.0,0.0,0.0,0.0,211.096358,8.106,False,A,151315
1,1,2010-02-12,1641957.44,73,1,38.51,2.548,0.0,0.0,0.0,0.0,0.0,211.242170,8.106,True,A,151315
2,1,2010-02-19,1611968.17,145,1,39.93,2.514,0.0,0.0,0.0,0.0,0.0,211.289143,8.106,False,A,151315
3,1,2010-02-26,1409727.59,218,1,46.63,2.561,0.0,0.0,0.0,0.0,0.0,211.319643,8.106,False,A,151315


In [68]:
df_total_sales.to_csv("../data/wrangled_data/df_total_sales.csv")